# Redes Neuronales
## Fashion detector 

In [ ]:
# dependencias necesarias
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from sklearn.metrics import accuracy_score, confusion_matrix

from IPython.display import Image, display

# configuración para ocultar warnings
import warnings
warnings.filterwarnings('ignore')

# configuración para no expresar en notación científica
np.set_printoptions(suppress=True)

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

In [ ]:
# obtenemos las imágenes (x) y salidas/categorías (y) del dataset
# divididas en train y test
train, test = fashion_mnist.load_data()
(x_train, y_train) = train 
(x_test, y_test) = test

### 1) Análisis exploratorio sobre el conjunto de datos.  
Este es un dataset de 70000 imágenes en blanco y negro de 28x28 pixeles, de 10 categorías de prendas, divididas en un set de train con 60000 imágenes y otro de test de 10000. 

In [ ]:
print('Cantidad y tamaño de imágenes de train:')
x_train.shape  

In [ ]:
print('Cantidad y tamaño de imágenes de test:')
x_test.shape

In [ ]:
# clases de prendas
CLASES = ["T-shirt/top", "Trouser", "Pullover", 
          "Dress", "Coat", "Sandal", "Shirt",
          "Sneaker", "Bag", "Ankle boot"]

Ejemplos de la imágenes sin modificaciones:

In [ ]:
# función para mostrar imágenes
def mostrar_imagenes(entradas, salidas):
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(entradas[i], cmap=plt.cm.binary)
        plt.xlabel(CLASES[salidas[i]])
    plt.show()

mostrar_imagenes(x_train, y_train)

Las categorías de prendas son las siguientes:

In [ ]:
for clase in CLASES:
    print(clase)

A partir de los gráficos siguientes, podemos notar que el set de datos está completamente balanceado tanto en train como test. En el set de train, hay 6000 imágenes de cada tipo de prenda, mientras que en el set de test hay 1000 imágenes de cada uno.

In [ ]:
# función para contar y graficar la cantidad de prendas por tipo
def distribucion(salidas, titulo=''):
    CANTIDADES = [0,0,0,0,0,0,0,0,0,0]
    for salida in salidas:
        CANTIDADES[salida] += 1
        
    display(titulo)
    plt.pie(CANTIDADES, labels=CLASES, autopct="%0.1f %%")

In [ ]:
distribucion(y_train, 'Distribución de train')

In [ ]:
distribucion(y_test, 'Distribución de test')

### 2) Machine Learning.  
#### Modificación de las imágenes:  
##### - Modificar el tamaño 
En cuanto al tamaño de las imágenes, optamos por NO modificarlo, debido a que ya es lo suficientemente pequeño como para entrenar sin demorar demasiado.  
###### - Reescalar valores
Decidimos reescalar los valores de las imágenes entre 0 y 1, tanto en test como en train, ya que es necesario para que las redes funcionen correctamente. Esto se puede comprobar en los siguientes gráficos que muestran el rango de valores que posee una imagen del dataset antes y después de reescalar.

In [ ]:
# reescalamos los valores de las imágenes
x_train_r = x_train / 255.0
x_test_r = x_test / 255.0

In [ ]:
plt.figure()
plt.title('Antes de reescalar')
plt.imshow(x_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
plt.figure()
plt.title('Después de escalar')
plt.imshow(x_train_r[0])
plt.colorbar()
plt.grid(False)
plt.show()

*Algunas funciones para graficar:*

In [ ]:
# función para graficar la curva de aprendizaje
def curva_aprendizaje(historial):
    plt.plot(historial.history['accuracy'], label='train')
    plt.plot(historial.history['val_accuracy'], label='test')
    plt.title('Accuracy over train epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
# función para graficar la matriz de confusión
def matriz_confusion(modelo, dt_x, dt_y, title=''):
    predictions = modelo.predict(dt_x)
    pred_label = [np.argmax(i) for i in predictions]
    labels = dt_y
    
    conf_matrix = confusion_matrix(labels, pred_label)

    ax = sns.heatmap(conf_matrix, 
                cmap='Blues', 
                xticklabels=CLASES, 
                yticklabels=CLASES,
                annot=True,
                fmt='d')

    plt.xlabel('Predicted class') 
    plt.ylabel('True class') 
    
    print(title)
    plt.show()

#### ENTRENAMIENTO Y EVALUACIÓN DE MODELOS:  
Para este análisis decidimos definir y evaluar diversas redes neuronales.

En un principio definimos una red MLP con determinados parámetros, y en base a ella, probamos otras redes a las cuales les fuimos modificando diversas características como cantidad de capas y de neuronas, cantidad de épocas, tamaño del batch, tipo de función de activación, nivel de dropout, etc. 

Luego a la primer red MLP le agregamos una capa convolucional con ciertas características y así poder probar diversas redes de tipo convolucional, cambiando cantidad de filtros, tamañano del kernel, strides, cantidad de capas convolucionales, padding, entre otras cosas.

##### Redes Neuronales 1, 1-a y 1-b:
La red 1 obtuvo valores de accuracy por encima del 70%, tanto para train como para test, ya en la primera época. Este valor fue aumentando hasta alcanzar un accuracy superior al 90% para train y al 87% para test en la última época.  
Además, al observar la curva de aprendizaje, podemos corroborar que no hay sobreentrenamiento, ya que la diferencia del accuracy en train y test, es tan solo del 3% aproximadamente. Sin embargo, a medida que pasan las épocas, estas líneas parecen ir separándose, por lo cual entrenamos una red (1-a) con las mismas características, pero con 80 épocas. Tras analizar la curva de aprendizaje de esta última, comprobamos que el error creció y que la distancia entre las líneas de train y test continúa aumentando, lo cual, si bien sube más los valores del accuracy en test, podría llegar a generar algo de sobreentrenamiento si continuamos agregando épocas.  
También decidimos probar a entrenar la red con un batch más pequeño (1-b) y evaluar las medidas obtenidas, pero notamos que no hubo grandes modificaciones en el accuracy. En general, de los 3 casos, hasta ahora, el mejor es el primero, ya que se obtuvo un accuracy mayor en el set de test.  
Con respecto a las matrices de confusión, podemos ver que, son bastante similares en los 3 casos. En general, si bien hay muchos aciertos, también hay predicciones erróneas, pero en las clases de prendas que son similares (remera y camisa, botas y zapatillas, por ejemplo), lo cual tiene sentido.

##### - Red Neuronal 1
* Tipo: MLP.
* Capas: 3 densas, con 20, 20 y 10 neuronas en ese orden.
* Dropout: no aplica.
* Función de activación: 'tanh' en la primeras capas y 'softmax' en la de salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_1 = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_1.summary()

In [ ]:
history_mlp_1 = model_mlp_1.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_1)

In [ ]:
matriz_confusion(model_mlp_1, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_mlp_1, x_test_r, y_test, 'Matriz de confusión - Test')

##### - Red Neuronal 1-a
* Tipo: MLP.
* Capas: 2 densas, con 20, 20 y 10 neuronas en ese orden.
* Dropout: no aplica.
* Función de activación: 'tanh' en la primer capa y 'softmax' en la de salida.
* **Épocas: 80.**
* Tamaño del batch: 250.

In [ ]:
model_mlp_1a = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_1a.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_1a.summary()

In [ ]:
history_mlp_1a = model_mlp_1a.fit(
    x_train_r,
    y_train,
    epochs=80,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_1a)

In [ ]:
matriz_confusion(model_mlp_1a, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_mlp_1a, x_test_r, y_test, 'Matriz de confusión - Test')

##### - Red Neuronal 1-b
* Tipo: MLP.
* Capas: 2 densas, con 20, 20 y 10 neuronas en ese orden.
* Dropout: no aplica.
* Función de activación: 'tanh' en la primer capa y 'softmax' en la de salida.
* Épocas: 25.
* **Tamaño del batch: 125.**

In [ ]:
model_mlp_1b = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_1b.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_1b.summary()

In [ ]:
history_mlp_1b = model_mlp_1b.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=125,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_1b)

In [ ]:
matriz_confusion(model_mlp_1b, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_mlp_1b, x_test_r, y_test, 'Matriz de confusión - Test')

##### Redes Neuronales 2, 2-a y 2-b:
Para la red 2, mantuvimos los valores de la red 1, a excepción del número de capas y neuronas, que decidimos aumentar. Tras el entrenamiento, esta obtuvo una diferencia mayor en el accuracy con respecto a la red 1 en ambos sets. Además, podemos ver que el error, para esta red con más parámetros, disminuyó.  
Tras modificar la cantidad de épocas en el entrenamiento (red 2-a), también pudimos comprobar que, si bien aumentaba la métrica en train, esta comenzaba a alejarse de los valores de test, confirmando nuevamente que si continuáramos agregando épocas, podría llegar a sobreentrenar.  
Por otra parte, decidimos probar una red con mucha más cantidad de capas y neuronas (2-b). Tras entrenarla, comprobamos que tan solo en 10 épocas el accuracy llegaba a valores del 10%, lo cual indica que tener una gran cantidad de parámetros no es bueno, tal y como vimos en la teoría. Esto también se puede comprobar en las matrices de confusión, ya que la red solo predice Shirt.

##### - Red Neuronal 2  
* Tipo: MLP.
* **Capas: 10 densas, con 60, 60, 60, 60, 40, 40, 40, 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la de salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_2 = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),   
    Dense(40, activation='tanh'),
    Dense(40, activation='tanh'),
    Dense(40, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_2.summary()

In [ ]:
history_mlp_2 = model_mlp_2.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_2)

In [ ]:
matriz_confusion(model_mlp_2, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_mlp_2, x_test_r, y_test, 'Matriz de confusión - Test')

##### - Red Neuronal 2-a  
* Tipo: MLP.
* **Capas: 10 densas, con 60, 60, 60, 60, 40, 40, 40, 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la de salida.
* **Épocas: 80.**
* Tamaño del batch: 250.

In [ ]:
model_mlp_2a = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),   
    Dense(40, activation='tanh'),
    Dense(40, activation='tanh'),
    Dense(40, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_2a.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_2a.summary()

In [ ]:
history_mlp_2a = model_mlp_2a.fit(
    x_train_r,
    y_train,
    epochs=80,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_2a)

In [ ]:
matriz_confusion(model_mlp_2a, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_mlp_2a, x_test_r, y_test, 'Matriz de confusión - Test')

##### - Red Neuronal 2-b  
* Tipo: MLP.
* **Capas: 31 densas, 3 de 200 neuronas, 7 de 120, 6 de 100, 5 de 80, 4 de 60, 3 de 40, 2 de 20 y la última de 10 neuronas, en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la de salida.
* **Épocas: 10.**
* Tamaño del batch: 250.

In [ ]:
model_mlp_2b = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(200, activation='tanh'),
    Dense(200, activation='tanh'),
    Dense(200, activation='tanh'),
    Dense(120, activation='tanh'),
    Dense(120, activation='tanh'),
    Dense(120, activation='tanh'),
    Dense(120, activation='tanh'),
    Dense(120, activation='tanh'),
    Dense(120, activation='tanh'),
    Dense(120, activation='tanh'),
    Dense(100, activation='tanh'),
    Dense(100, activation='tanh'),
    Dense(100, activation='tanh'),
    Dense(100, activation='tanh'),
    Dense(100, activation='tanh'),
    Dense(100, activation='tanh'),
    Dense(80, activation='tanh'),
    Dense(80, activation='tanh'),
    Dense(80, activation='tanh'),
    Dense(80, activation='tanh'),
    Dense(80, activation='tanh'),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),
    Dense(60, activation='tanh'),   
    Dense(40, activation='tanh'),
    Dense(40, activation='tanh'),
    Dense(40, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_2b.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_2b.summary()

In [ ]:
history_mlp_2b = model_mlp_2b.fit(
    x_train_r,
    y_train,
    epochs=10,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_2b)

In [ ]:
matriz_confusion(model_mlp_2b, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_mlp_2b, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 3:  
En esta red, con respecto a la primera, decidimos modificar la función de activación de las capas ocultas, reemplazando ‘tanh’ por ‘relu’, para evaluar si genera alguna diferencia. Tras el entrenamiento comprobamos que este cambio no generó mejores resultados, ya que el accuracy, tanto en train como en test, se redujo. También vimos que aumentó el error.    
* Tipo: MLP.
* Capas: 3 densas, con 20, 20 y 10 neuronas en ese orden.
* Dropout: no aplica.
* **Función de activación: 'relu' en la primeras capas y 'softmax' en la de salida.**
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_3 = Sequential([
    
    Flatten(input_shape=(28, 28, 1)),
    Dense(20, activation='relu'),
    Dense(20, activation='relu'),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_3.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_3.summary()

In [ ]:
history_mlp_3 = model_mlp_3.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_3)

In [ ]:
matriz_confusion(model_mlp_3, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_mlp_3, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 4:  
Para esta red, mantuvimos los valores de la red 1, a excepción del dropout, ya que decidimos aplicarle un 30% y analizar los resultados. Luego de entrenar, verificamos que esta modificación empeoró los valores del accuracy tanto para train, como para test. También vimos que aumentó el error.  
* Tipo: MLP.
* Capas: 3 densas, con 20, 20 y 10 neuronas en ese orden.
* **Dropout: 30% en capas ocultas.**
* Función de activación: 'tanh' en la primeras capas y 'softmax' en la de salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_4 = Sequential([
    
    Flatten(input_shape=(28, 28, 1)),
    Dense(20, activation='relu'),
    Dropout(0.3),
    Dense(20, activation='relu'),
    Dropout(0.3),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_4.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_4.summary()

In [ ]:
history_mlp_4 = model_mlp_4.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_4)

In [ ]:
matriz_confusion(model_mlp_4, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_mlp_4, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 5:  
Esta red presenta las mismas características que la red 1, pero con la diferencia de que ahora incluye una capa convolucional de 4 filtros de 2x2 y stride 1. Esto hizo que el accuracy sea el mayor hasta ahora, con respecto a las demás redes analizadas, es decir que mejora el valor de la métrica. Además, también hizo que se redujera el error con respecto a dichas redes.  
* **Tipo: Convolucional.**
* **Capas: 1 convolucional con 4 filtros de 2x2 y stride 1, un max pooling de 2x2 y 3 densas, con 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_conv_1 = Sequential([
    Convolution2D(input_shape=(28, 28, 1), filters=4, kernel_size=(2, 2), strides=1, activation='tanh'), 
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_conv_1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_conv_1.summary()

In [ ]:
history_conv_1 = model_conv_1.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_conv_1)

In [ ]:
matriz_confusion(model_conv_1, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_conv_1, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 6:    
Basándonos en la red 5, decidimos probar una red que modifique la cantidad de filtros de la capa de convolución, que ahora pasará de 4 a 8. Este cambió generó un aumento del accuracy con  respecto a la capa anterior, por lo cual consideramos que tener más filtros, en este caso, podría llegar a ser una mejor opción. Además, también hizo que se redujera considerablemente el error, inclusive más que la red anterior.  
* Tipo: Convolucional.
* **Capas: 1 convolucional con 8 filtros de 2x2 y stride 1, un max pooling de 2x2 y 3 densas, con 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_conv_2 = Sequential([    
    Convolution2D(input_shape=(28, 28, 1), filters=8, kernel_size=(2, 2), strides=1, activation='tanh'), 
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_conv_2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_conv_2.summary()

In [ ]:
history_conv_2 = model_conv_2.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_conv_2)

In [ ]:
matriz_confusion(model_conv_2, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_conv_2, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 7: 
Basándonos en la red 5, también decidimos probar una red que modifique el tamaño de los filtros de la capa de convolución, que ahora pasará de 4x4 a 8x8. Este cambió bajó, en muy pequeña medida, el valor del accuracy con respecto a las anteriores, por lo que podríamos decir que esta modificación, no aporta grandes mejoras. También podemos mencionar que, si bien, reduce un poco el error, este valor no es significativo respecto al error de las demás redes.  
* Tipo: Convolucional.
* **Capas: 1 convolucional con 4 filtros de 8x8 y stride 1, un max pooling de 2x2 y 3 densas, con 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_conv_3 = Sequential([
    Convolution2D(input_shape=(28, 28, 1), filters=4, kernel_size=(8, 8), strides=1, activation='tanh'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_conv_3.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_conv_3.summary()

In [ ]:
history_conv_3 = model_conv_3.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_conv_3)

In [ ]:
matriz_confusion(model_conv_3, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_conv_3, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 8:  
A partir de la red 5, también decidimos probar una red que modifique el stride de la capa de convolución, que ahora pasará de 1 a 2. Este cambió no aportó una mejora significativa con respecto al accuracy de las redes anteriores, ya que puntualmente, presenta casi el mismo valor que la red 1. También podemos mencionar que, si bien, reduce un poco el error, este valor no es significativo respecto al error de las demás redes.  
* Tipo: Convolucional.
* **Capas: 1 convolucional con 4 filtros de 2x2 y stride 2, un max pooling de 2x2 y 3 densas, con 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_conv_4 = Sequential([
    Convolution2D(input_shape=(28, 28, 1), filters=4, kernel_size=(2, 2), strides=2, activation='tanh'), 
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_conv_4.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_conv_4.summary()

In [ ]:
history_conv_4 = model_conv_4.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_conv_4)

In [ ]:
matriz_confusion(model_conv_4, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_conv_4, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 9:  
Basándonos en la red 5, también decidimos probar una red que rellene la entrada con 0, es decir, utilizar padding. Este cambió mejoró, en muy pequeña medida, el valor del accuracy con respecto a la red 1 pero no con respecto a todas las anteriores, por lo que podríamos decir que esta modificación, no aporta grandes mejoras. También podemos mencionar que reduce considerablemente el error, con respecto al de las demás redes, pero no más que la red 6.  
* Tipo: Convolucional.
* **Capas: 1 convolucional con 4 filtros de 2x2, stride 1 y padding, un max pooling de 2x2 y 3 densas, con 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_conv_5 = Sequential([
    Convolution2D(input_shape=(28, 28, 1), filters=4, kernel_size=(2, 2), strides=1, padding='same', activation='tanh'), 
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_conv_5.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_conv_5.summary()

In [ ]:
history_conv_5 = model_conv_5.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_conv_5)

In [ ]:
matriz_confusion(model_conv_5, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_conv_5, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 10:  
Quizás trabajar con una sola capa convolucional es poco, así que decidimos probar una red que agregue más de una capa de convolución. Ahora trabajaremos con 3 capas en lugar de 1. Estas tendrán las mismas características que la red 5. Tras el entrenamiento podemos ver que este cambio aportó una suba en el valor de la métrica con respecto a dicha red 5, por lo cual es un factor a tener en cuenta a la hora de “mejorar” una red. Además, también hizo que se redujera considerablemente el error, casi al nivel de la red 6.  
* Tipo: Convolucional.
* **Capas: 3 convolucionales con 4 filtros de 2x2 y stride 1, un max pooling de 2x2 y 3 densas, con 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_conv_6 = Sequential([
    Convolution2D(input_shape=(28, 28, 1), filters=4, kernel_size=(2, 2), strides=1, activation='tanh'),
    Convolution2D(filters=4, kernel_size=(2, 2), strides=1, activation='tanh'), 
    Convolution2D(filters=4, kernel_size=(2, 2), strides=1, activation='tanh'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_conv_6.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_conv_6.summary()

In [ ]:
history_conv_6 = model_conv_6.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_conv_6)

In [ ]:
matriz_confusion(model_conv_6, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_conv_6, x_test_r, y_test, 'Matriz de confusión - Test')

##### Red Neuronal 11:  
Para esta red, decidimos aplicar aquellos cambios que consideramos que, por los análisis anteriores, mejoraban en mayor medida, los valores del accuracy y del error, como por ejemplo, agregar más capas y aumentar la cantidad de filtros por capa. Estos cambios, confirmaron que aplicarlos, genera mejores resultados, ya que de todas las redes probadas, es la que mayor valor de accuracy obtuvo. Aunque cabe aclarar que no es una diferencia tan grande, sobre todo con respecto a la red 6. También podemos mencionar que redujo considerablemente el error, pero no más que dicha red 6. Podemos decir que con solo agregar más filtros podría ser suficiente para obtener buenos valores. Con respecto a la curva de aprendizaje, podemos ver que a medida que avanzan las épocas, las líneas de train y test se separan cada vez más, por lo cual si agregaríamos más épocas, el modelo podría llegar a sobreentrenar.  
* Tipo: Convolucional.
* **Capas: 4 convolucionales, 1 con 16 filtros y 3 con 8 filtros de 2x2 y stride 1, un max pooling de 2x2 y 3 densas, con 20, 20 y 10 neuronas en ese orden.**
* Dropout: no aplica.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_conv_7 = Sequential([
    Convolution2D(input_shape=(28, 28, 1), filters=16, kernel_size=(2, 2), strides=1, activation='tanh'),
    Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'), 
    Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'),
    Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_conv_7.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_conv_7.summary()

In [ ]:
history_conv_7 = model_conv_7.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_conv_7)

In [ ]:
matriz_confusion(model_conv_7, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_conv_7, x_test_r, y_test, 'Matriz de confusión - Test')

#### AUMENTACIÓN DE DATOS:
Para probar esta técnica elegimos el modelo que mayor valor de accuracy obtuvo en train, es decir, la red 11, que agregaba más capas convolucionales y más filtros. Por otra parte, también decidimos aplicarlo en la primer red creada, es decir la 1, para verificar si generaba mejoras.  
Para modificar las imágenes, alteramos parámetros como el ángulo de rotación, nivel de desplazamiento horizontal y vertical, brillo, espejar horizontalmente la imagen, y nivel de aumento.  
Una vez hecho esto, agregamos dichas imágenes al dataset original para generar el aumento de datos. Ahora pasaríamos de entrenar con 60000 imágenes a entrenar con 120000.

In [ ]:
# generamos un dataset de train con imagénes alteradas
# se obtiene en formato de tensor
data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.5, 1.5),
    horizontal_flip=True,
    vertical_flip=False,
    zoom_range=0.1,
)

train_alterado = data_generator.flow(
    x_train_r.reshape(60000,28,28,1),
    y_train,
    batch_size = 60000,
    shuffle=False)

In [ ]:
# utilizamos el mismo generador (pero sin modificaciones)
# para obtener el dataset en formato de tensor
data_no_generator = ImageDataGenerator()

train_no_alterado = data_no_generator.flow(
    x_train_r.reshape(60000,28,28,1),
    y_train,
    batch_size = 60000,
    shuffle=False)

In [ ]:
# separamos los tensores en imágenes y etiquetas
imagenes_a, etiquetas_a = train_alterado.next()
imagenes_n, etiquetas_n = train_no_alterado.next()

Podemos ver algunos ejemplos de las imágenes modificadas:

In [ ]:
mostrar_imagenes(imagenes_a, etiquetas_a)

In [ ]:
# concatenamos las imágenes originales con las modificadas
train_ampliado_imagenes = np.concatenate((
    imagenes_n,
    imagenes_a
))

In [ ]:
# concatenamos los labels de las imágenes originales con los
# de las imágenes modificadas
train_ampliado_etiquetas = np.concatenate((
    etiquetas_n,
    etiquetas_a
))

In [ ]:
# generamos una red con las mismas características que la nro. 11
model_conv_8 = Sequential([
    Convolution2D(input_shape=(28, 28, 1), filters=16, kernel_size=(2, 2), strides=1, activation='tanh'),
    Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'), 
    Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'),
    Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_conv_8.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_conv_8.summary()

In [ ]:
history_conv_8 = model_conv_8.fit(
    train_ampliado_imagenes,
    train_ampliado_etiquetas,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_conv_8)

In [ ]:
# generamos una red con las mismas características que la nro. 1
model_mlp_12 = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax'),
])

model_mlp_12.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_12.summary()

In [ ]:
history_mlp_12 = model_mlp_12.fit(
    train_ampliado_imagenes,
    train_ampliado_etiquetas,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_12)

Tras entrenar ambos modelos con el dataset aumentado, pudimos comprobar que, en este caso, la técnica no aportó mejoras, ya que el valor del accuracy obtenido en ambos modelos, fue mayor cuando se entrenó con el dataset original. Incluso el nivel de error se mantuvo más alto en los modelos con los dataset aumentados.    
Podemos suponer que esto se debe a que quizás las alteraciones sobre las imágenes fueron un tanto excesivas o no las adecuadas para estas imágenes.  

#### TRANFER LEARNING - VGG16:     
Uno de los modelos de alto rendimiento para el reconocimiento de imágenes que se pueden descargar y utilizar como base para el reconocimiento de imágenes es el VGG16. Este es el modelo que decidimos usar para aplicar la transferencia de aprendizaje.  
VGG16 contiene varios bloques de capas de convolución de filtros de 3x3 con stride de 1 y max pooling de 2x2 con stride 2. Al final, tiene 2 capas densas, seguidas de softmax para la salida. El 16 en VGG16 se refiere a que tiene 16 capas que tienen pesos.   
El modelo logra una precisión de prueba del 92,7% entre los 5 primeros datasets en ImageNet (conjuntos de datos de más de 14 millones de imágenes pertenecientes a 1000 clases).   
Para utilizar este modelo, reemplazamos las capas densas y de salida del mismo, por otras capas que nos servirán para predecir las imágenes de nuestro dataset. Manteniendo el peso de las capas convolucionales, entrenamos las capas agregadas con las imágenes de train, previamente alteradas para adaptarse a VGG.

In [ ]:
# utilizamos el generador utilizado anteriormente (sin cambios)
# para obtener el dataset de test en formato de tensor
test_no_alterado = data_no_generator.flow(
    x_test_r.reshape(10000,28,28,1),
    y_test,
    batch_size = 10000,
    shuffle=False)

# separamos el tensor en imágenes y etiquetas
imagenes_n_t, etiquetas_n_t = test_no_alterado.next()

In [ ]:
# modificamos las imágenes para que tengan 3 canales (rgb)
# de manera que sean toleradas por VGG16
x_train_color = tf.image.grayscale_to_rgb(tf.convert_to_tensor(imagenes_n))
x_test_color = tf.image.grayscale_to_rgb(tf.convert_to_tensor(imagenes_n_t))

# modificamos el tamaño de las imágenes al mínimo tolerado por VGG16
x_train_color = tf.image.resize(x_train_color, [32,32])
x_test_color = tf.image.resize(x_test_color, [32,32])

In [ ]:
print('Nuevo formato de las imágenes:')
x_train_color[0].shape

In [ ]:
# pre procesamos las imágenes para que sean utilizadas por VGG16
train_i = preprocess_input(x_train_color) 
test_i = preprocess_input(x_test_color)

In [ ]:
# descargamos VGG16 con sus pesos correspondientes
# descartando las capas densas
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(32,32,3))

# frizzamos las capas convolucionales para preservar
# los pesos durante el entrenamiento
base_model.trainable = False

base_model.summary()

In [ ]:
# creamos un modelo que tome como base el VGG16 descargado
# adicionando nuevas capas densas
model_tl = Sequential([
    base_model,
    Flatten(),
    Dense(20, activation='tanh'),
    Dense(20, activation='tanh'),
    Dense(len(CLASES), activation='softmax')
])

model_tl.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_tl.summary()

In [ ]:
history_tl = model_tl.fit(
    x_train_color,
    etiquetas_n,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_color, etiquetas_n_t)
)

In [ ]:
curva_aprendizaje(history_tl)

In [ ]:
matriz_confusion(model_tl, x_train_r, y_train, 'Matriz de confusión - Train')
matriz_confusion(model_tl, x_test_r, y_test, 'Matriz de confusión - Test')

Tras el entrenamiento comprobamos que el modelo obtuvo un accuracy en el set de test bastante bueno, pero no superior a los modelos trabajados anteriormente. Creemos que un factor que podría influenciar su aprovechamiento es que VGG trabaja con imágenes a color, mientras que nuestro dataset está en escala de grises. Si bien hicimos las transformaciones necesarias para que respete este formato, algunos filtros que pudiesen haber sacado información relacionada a los colores podrían no estarse aprovechando.

### 3) Conclusiones.   
AGREGAR CONCLUSIÓN GENERAL

#### Desempeño del modelo por clase:  
Para evaluar el desempeño de un modelo por clase, elegimos la red 11, es decir, la misma que utilizamos para hacer aumentación de datos, ya que era la que mayor valor de accuracy obtenía.  
A partir de la matriz de confusión del modelo, podemos ver que este tiene un muy buen desempeño, mantiendo la mayoría de las predicciones en la categoría correcta, exceptuando casos en los que se confunde prendas que son bastante similares como: T-shirt/top y Shirt, Pullover y Coat, Coat y Shirt, entre otros.

In [ ]:
matriz_confusion(model_conv_7, x_test_r, y_test, 'Matriz de confusión - Test')

#### Aciertos y desaciertos:

In [ ]:
# obtenemos las predicciones de test de la red 11 
predictions = model_conv_7.predict(x_test_r)

In [ ]:
# funciones para graficar los ejemplos y
# mostrar los mayores porcentajes de confianza de las clases
# diferenciando la acertada de las que no
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(CLASES[predicted_label],
                                100*np.max(predictions_array),
                                CLASES[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# función para mostrar casos de aciertos y desaciertos
def mostrar_aciertos_desaciertos(clase, acierto):
    count = 0
    for i, prediction in enumerate(predictions):
        if(acierto==1):
            if(CLASES[np.argmax(prediction)]==CLASES[y_test[i]]):
                if(clase==CLASES[np.argmax(prediction)]):
                    count+=1            
                    plt.figure(figsize=(4,2))
                    plt.subplot(1,2,1)
                    plot_image(i, prediction, y_test, x_test_r)
                    plt.subplot(1,2,2)
                    plot_value_array(i, prediction,  y_test)
                    plt.show()
        else:
            if(CLASES[np.argmax(prediction)]!=CLASES[y_test[i]]):
                if(clase==CLASES[y_test[i]]):
                    count+=1            
                    plt.figure(figsize=(4,2))
                    plt.subplot(1,2,1)
                    plot_image(i, prediction, y_test, x_test_r)
                    plt.subplot(1,2,2)
                    plot_value_array(i, prediction,  y_test)
                    plt.show()
        if(count==2):
            break           

###### T-shirt/top

In [ ]:
mostrar_aciertos_desaciertos('T-shirt/top', 0)

In [ ]:
mostrar_aciertos_desaciertos('T-shirt/top', 1)

###### Trouser

In [ ]:
mostrar_aciertos_desaciertos('Trouser', 0)

In [ ]:
mostrar_aciertos_desaciertos('Trouser', 1)

###### Pullover

In [ ]:
mostrar_aciertos_desaciertos('Pullover', 0)

In [ ]:
mostrar_aciertos_desaciertos('Pullover', 1)

###### Dress

In [ ]:
mostrar_aciertos_desaciertos('Dress', 0)

In [ ]:
mostrar_aciertos_desaciertos('Dress', 1)

###### Coat

In [ ]:
mostrar_aciertos_desaciertos('Coat', 0)

In [ ]:
mostrar_aciertos_desaciertos('Coat', 1)

###### Sandal

In [ ]:
mostrar_aciertos_desaciertos('Sandal', 0)

In [ ]:
mostrar_aciertos_desaciertos('Sandal', 1)

###### Shirt

In [ ]:
mostrar_aciertos_desaciertos('Shirt', 0)

In [ ]:
mostrar_aciertos_desaciertos('Shirt', 1)

###### Sneaker

In [ ]:
mostrar_aciertos_desaciertos('Sneaker', 0)

In [ ]:
mostrar_aciertos_desaciertos('Sneaker', 1)

###### Bag

In [ ]:
mostrar_aciertos_desaciertos('Bag', 0)

In [ ]:
mostrar_aciertos_desaciertos('Bag', 1)

###### Ankle boot

In [ ]:
mostrar_aciertos_desaciertos('Ankle boot', 0)

In [ ]:
mostrar_aciertos_desaciertos('Ankle boot', 1)

#### Casos reales:

Decidimos probar dicha red 11 con otras imágenes de prendas, fuera del dataset. Como podemos ver las predicciones no fueron muy buenas, ya que de 4 casos, solo acertó un caso y con una probabilidad baja.  
Esto puede indicarnos que el modelo funciona bien solo con las imágenes del dataset, es decir que de alguna manera, está sobreentrenando.

In [ ]:
# función para mostrar una imagen de ropa y predecir su tipo
def mostrar_predecir(image_path):
    image_array = img_to_array(load_img(image_path, grayscale=True, target_size=(28, 28)))
    inputs = np.array([image_array])
    predictions = model_conv_7.predict(inputs)
    display(Image(image_path, width=150))
    print("Prediction:", CLASES[np.argmax(predictions)])
    print("Prediction detail:", predictions)

In [ ]:
mostrar_predecir("./ropa/Shirt.jpg")

In [ ]:
mostrar_predecir("./ropa/Ankle_boot.jpg")

In [ ]:
mostrar_predecir("./ropa/Bag.jpg")

In [ ]:
mostrar_predecir("./ropa/Trouser.jpg")